##  implement of simply GA & BPNN  ##

In [85]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

%matplotlib inline

In [86]:
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s

In [87]:
def layer_sizes(X, Y):
    
    n_x = X.shape[0] # size of input layer
    n_h = 4 # 硬编码为4
    n_y = Y.shape[0] # size of output layer
    
    return (n_x, n_h, n_y)

def initialize_parameters(n_x, n_h, n_y):
    
#     np.random.seed(2)     
    W1 = np.random.randn(n_h,n_x) * 0.01
    b1 = np.random.randn(n_h,1)
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.random.randn(n_y,1)
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

def forward_propagation(X, parameters):
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2 , A1) + b2
    A2 = sigmoid(Z2)
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

def compute_cost(A2, Y):
    
    m = Y.shape[1]
    logprobs = np.multiply(np.log(A2),Y) + np.multiply((1 - Y), np.log(1 - A2))
    cost = - np.sum(logprobs) / m
    cost = float(np.squeeze(cost))
    assert(isinstance(cost, float))
    
    return cost

def backward_propagation(parameters, cache, X, Y):
    
    m = X.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    A1 = cache["A1"]
    A2 = cache["A2"]
    dZ2= A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

def update_parameters(parameters, grads, learning_rate = 1.2):
    
    W1,W2 = parameters["W1"],parameters["W2"]
    b1,b2 = parameters["b1"],parameters["b2"]
    dW1,dW2 = grads["dW1"],grads["dW2"]
    db1,db2 = grads["db1"],grads["db2"]
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

def nn_model(X, Y, n_h, num_iterations = 10000, learning_rate = 0.5, print_cost=False):
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters
    parameters = initialize_parameters(n_x,n_h,n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Loop (gradient descent)
    cost_list = []
    parameters_list =[]
    
    for i in range(0, num_iterations):
        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y)
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters,grads,learning_rate = 0.5)
        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if i< 100:
          cost_list.append(cost)
          parameters_list.append(parameters )
    return parameters, cost_list, parameters_list

def predict(parameters, X):

    A2, cache = forward_propagation(X,parameters)
    predictions = np.round(A2)
    
    return predictions

def model(X_train, Y_train, X_test, Y_test, n_h, num_iterations,learning_rate, print_cost):
    
    parameters = nn_model(X_train, Y_train, n_h, num_iterations,learning_rate, print_cost)
    # Predict test/train set examples
    Y_prediction_test = predict(parameters, X_test)
    Y_prediction_train = predict(parameters, X_train)
    
    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    
    d = {"n_h": n_h,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "parameters" : parameters,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d
    
def net(X, Y, n_h, num_iterations,learning_rate, print_cost=False): 
    parameters , cost_list, parameters_list = nn_model(X, Y, n_h, num_iterations ,learning_rate, print_cost)      
    bestcostindex = cost_list.index(min(cost_list ))
    parameters_ga = parameters_list.index(min(parameters_list))
    predictions_ga = predict(parameters_ga, X)
    print('Accuracy: %d' % float((np.dot(Y,predictions_ga.T) + np.dot(1-Y,1-predictions_ga.T))/float(Y.size)*100) + '%')

    return predictions_ga 

In [137]:
#  functions

def fit(x, n_x, n_h, n_y,X, Y):
    """
    该函数用来计算适应度值
    
    Arguments:
    x     —  个体
    n_x   —  输入层节点数
    n_h   —  隐含层节点数
    n_y   —  输出层的点数
    X     —  训练输入数据
    Y     —  训练输出数据
    
    Return：
    Fitness  —  个体适应度值
    """
    # 提取
    w1 = x[0 : n_x * n_h]
    b1 = x[n_x * n_h : n_x * n_h + n_h]
    w2 = x[n_x * n_h + n_h : n_x * n_h + n_h + n_h * n_y]
    b2 = x[n_x * n_h + n_h + n_h * n_y : n_x * n_h + n_h + n_h * n_y + n_y]
    
    # 网络进化参数
    num_i = 20
    l_r = 0.1
    
    # 网络权值赋值
    W1 = np.array(w1).reshape(n_h, n_x)
    b1 = np.array(b1).reshape(n_h, 1)
    W2 = np.array(w2).reshape(n_y, n_h)
    b2 = np.array(b2).reshape(n_y, 1)
    
    # 网络训练
    
    predictions_ga = net(X, Y, n_h, num_iterations = num_i,learning_rate= l_r, print_cost=False)
    Fitness = np.sum(np.abs(Y,predictions_ga))
    return Fitness

In [89]:
def select(individuals, sizepop):
    """
    本函数对每一代种群中的染色体进行选择，以进行后面的交叉和变异
    
    Arguments:
    individuals — 种群信息
    sizepop   —   种群规模

    Return:
    ret  — 经过选择后的种群
    """
    # 根据个体适应度值进行排序
    fitn = np.array(individuals["fitness"])  # fitness类型为list
    fitness1_array = 10. / fitn
    sumfitness_array = np.sum(fitness1_array)
    sumf_array = fitness1 /sumfitness_array
    sumf = list(sumf_array)
    
    index=[] 
    for i in range(sizepop):   # 转sizepop次轮盘
        pick = np.random.rand(1)[0]
        while pick == 0.:
            pick = np.random.rand(1)[0]       
        for j in range(sizepop): 
            pick = pick-sumf[j]
            if pick < 0:
                index = index.append(j)          
                break 
    individuals["chrom"] = [individuals["chrom"][i] for i in index]   # chrom类型为list，元素也为list
    individuals["fitness"] = [individuals["fitness"][i] for i in index]
    ret = individuals
    
    return ret

In [134]:
def code(lenchrom,bound):
    """
    本函数将变量编码成染色体，用于随机初始化一个种群
    lenchrom    —    染色体长度
    bound   —     变量的取值范围
    
    ret    —   染色体的编码值
    """
    flag = 0
    while flag == 0:
        pick = np.random.rand(1,len(lenchrom))
        bound_array = np.array(bound)
        ret_array = bound_array[:,0]+ (bound_array[:,1]-bound_array[:,0]) * pick;  # 线性插值，编码结果以实数向量存入ret中
        ret = ret_array.tolist()
        flag = test(lenchrom,bound,ret)    # 检验染色体的可行性
    
    return ret

In [91]:
def decode(code):
    """
    本函数对染色体进行解码
    
    Arguments:
    code   —   编码值

    Return:
    ret     —    染色体的解码值
    """
    #  float coding
    ret=code  # 解码结果就是编码结果（实数向量），存入ret中
    return ret    

In [92]:
def test(lenchrom,bound,code):
    """
    检验染色体的可行性
    
    Arguments:
    lenchrom   —  染色体长度
    bound     —   变量的取值范围
    code     —    染色体的编码值

    Return:
    
    """
    x = code   # 先解码
    flag = 1
    
#     if (x(1)<0)&&(x(2)<0)&&(x(3)<0)&&(x(1)>bound(1,2))&&(x(2)>bound(2,2))&&(x(3)>bound(3,2)):
#         flag=0
    return flag

In [93]:
def cross(pcross,lenchrom,chrom,sizepop,bound):
    """
    本函数完成交叉操作
    
    Arguments:
    pcross     —  交叉概率
    lenchrom   —  染色体的长度
    chrom      —  染色体群
    sizepop    —  种群规模
    
    Return:
    ret   — 交叉后的染色体
    """
    for i in range(sizepop):
        # 随机选择两个染色体进行交叉
        pick = np.random.rand(1,2)
        while pick[0][0] * pick[0][1] == 0.:
            pick = np.random.rand(1,2)
        index_array = np.ceil(pick * sizepop)
        index = list(index_array[0])
        # 交叉概率决定是否进行交叉
        pick = np.random.rand(1)[0]
        while pick == 0.:
            pick = np.random.rand(1)[0]
        if pick > pcross:
            continue
        flag = 0
        while flag == 0:
            # 随机选择交叉位
            pick = np.random.rand(1)[0]
            while pick == 0.:
                pick = np.random.rand(1)[0]
            pos = np.int(np.ceil(pick * np.sum(lenchorm)))
            pick = np.random.rand(1)[0]    # 交叉开始
            v1 = chrom[np.int(index[0])][:pos]
            v2 = chrom[np.int(index[1])][:pos]
            chrom[np.int(index[0])][:pos] = pick*v2 + (1-pick)*v1;
            chrom[np.int(index[1])][:pos] = pick*v1 + (1-pick)*v2;   # 交叉结束
            flag1 = test(lenchrom,bound,chrom[index[0]])  # 检验染色体1的可行性
            flag2 = test(lenchrom,bound,chrom[index[1]])  # 检验染色体2的可行性
            if flag1 * flag2 == 0:
                flag=0
            else:
                flag=1

    ret = chrom
    return ret 

In [94]:
def mutation(pmutation,lenchrom,chrom,sizepop,num,maxgen,bound):
    """
    本函数完成变异操作
    pcorss                input  : 变异概率
    lenchrom              input  : 染色体长度
    chrom     input  : 染色体群
    sizepop               input  : 种群规模
    opts                  input  : 变异方法的选择
    pop                   input  : 当前种群的进化代数和最大的进化代数信息
    bound                 input  : 每个个体的上届和下届
    maxgen                input  ：最大迭代次数
    num                   input  : 当前迭代次数
    ret                   output : 变异后的染色体
    """
    for i in range(sizepop):
        # 随机选择一个染色体进行变异
        pick = np.random.rand(1)[0]
        while pick == 0.:
            pick = np.random.rand(1)[0]
        index = np.ceil(pick * sizepop)
        % 变异概率决定该轮循环是否进行变异
        pick = np.random.rand(1)[0]
        if pick > pmutation:
            continue
        flag = 0
        while flag == 0:
            # 变异位置
            pick = np.random.rand(1)[0]
            while pick == 0.:
                pick = np.random.rand(1)[0]
            pos = np.int(np.ceil(pick * np.sum(lenchorm)))
            pick = np.random.rand(1)[0]   # 变异开始 
            
            fg = np.power(((np.random.rand(1)[0])*(1-num/maxgen)),2)
            if pick > 0.5:
                chrom[i-1][pos] = chrom[i-1][pos]+(bound(pos,2)-chrom[i-1][pos])*fg
            else:
                chrom[i-1][pos] = chrom[i-1][pos]-(chrom[i-1][pos]-bound(pos,1))*fg
            # 变异结束
            flag = test(lenchrom,bound,chrom[i-1])  # 检验染色体的可行性

    ret = chrom;
    return ret

In [138]:
# loading the data(cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()
# Reshape the training and test data sets
train_set_x_flatten =train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_x_flatten =test_set_x_orig .reshape(test_set_x_orig.shape[0], -1).T
# standardize your dataset
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255

X_train = train_set_x
Y_train = train_set_y


# 节点个数
(n_x, n_h, n_y) = layer_sizes(X_train, Y_train)

# 遗传算法参数初始化
maxgen = 20             # 进化代数，即迭代次数
sizepop = 10            # 种群规模
pcross = [0.2]          # 交叉概率选择，0和1之间
pmutation = [0.1]       # 变异概率选择，0和1之间

# 节点总数
numsum = n_x * n_h + n_h + n_h * n_y + n_y
# print(numsum)
lenchrom = np.ones((1,numsum))[0].tolist()     
bound = (np.zeros((2,numsum))+ np.array([[-3],[3]])).T.tolist()    # 数据范围
#------------------------------------------------------种群初始化--------------------------------------------------------
individuals = {"fitness":list(np.zeros((1,sizepop))[0]), "chrom":np.zeros((sizepop, numsum)).tolist()}  # 将种群信息定义为一个字典
avgfitness=[]               # 每一代种群的平均适应度
bestfitness=[]             # 每一代种群的最佳适应度
bestchrom=[]                  # 适应度最好的染色体
# 初始化种群
for i in range(sizepop):
    # 随机产生一个种群
    individuals["chrom"][i] = code(lenchrom,bound) # 编码
    x = individuals["chrom"][i]
    # 计算适应度
    individuals["fitness"][i] = fit(x,n_x,n_h,n_y,X_train,Y_train)  # 染色体的适应度
FitRecord = np.zeros((maxgen,numsum)).tolist() 
# 找最好的染色体
bestfitness = min(individuals["fitness"])
bestindex = individuals["fitness"].index(bestfitness)
bestchrom = individuals["chrom"][bestindex]    # 最好的染色体
avgfitness = np.sum(individuals["fitness"]) / sizepop   # 染色体的平均适应度
#  记录每一代进化中最好的适应度和平均适应度
trace = np.zeros((maxgen + 1,2)).tolist()
trace[0] = [avgfitness,bestfitness] 
 
# 迭代求解最佳初始阀值和权值
# 进化开始
for i in range(maxgen):
    # 选择
    individuals = select(individuals,sizepop) 
    avgfitness = np.sum(individuals["fitness"])/sizepop 
    # 交叉
    individuals["chrom"] = cross(pcross,lenchrom,individuals["chrom"],sizepop,bound) 
    # 变异
    individuals["chrom"]=mutation(pmutation,lenchrom,individuals["chrom"],sizepop,i+1,maxgen,bound);
    
    # 计算适应度 
    for j in range(sizepop):
        x = individuals["chrom"][j] # 解码
        individuals["fitness"][j]=fit(x,n_x,n_h,n_y,net,X_train,Y_train)   
    
   # 找到最小和最大适应度的染色体及它们在种群中的位置
    newbestfitness = min(individuals["fitness"])
    newbestindex = individuals["fitness"].index(newbestfitness)
    worestfitness = max(individuals["fitness"])
    worestindex   = individuals["fitness"].index(worestfitness)                  
    #  代替上一次进化中最好的染色体
    if bestfitness > newbestfitness:
        bestfitness = newbestfitness
        bestchrom = individuals["chrom"][newbestindex]
    individuals["chrom"][worestindex] = bestchrom
    individuals["fitness"][worestindex] = bestfitness
    
    avgfitness = np.sum(individuals["fitness"])/sizepop
    
    trace[i+1] = [avgfitness ,bestfitness] # 记录每一代进化中最好的适应度和平均适应度
    FitRecord[i] = individuals["fitness"]
               


ValueError: cannot reshape array of size 49161 into shape (4,12288)

In [133]:
pick = np.random.rand(1,len(lenchrom))
print(pick)

[[0.18499804 0.69669084 0.36456881 ... 0.54323941 0.357635   0.3059789 ]]
